<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GenerateLocationData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker  

     |████████████████████████████████| 1.5 MB 3.9 MB/s 


In [18]:
import pandas as pd
import plotly.express as px
from faker import Faker
import numpy as np

In [21]:
fake = Faker()
rng=np.random.RandomState(123)


In [152]:
base = np.array((39.1057, -84.3671))
Nset=9
df_list = []
names = [fake.name_nonbinary() for _ in range(Nset)]
for _ in names:
    name = fake.name_nonbinary()
    startpos = base +rng.normal(scale=0.05, size =2)
    N=rng.randint(5,20)
    timestart = pd.Timestamp.now(tz='EST')-pd.to_timedelta(rng.randint(50,100),unit='D')
    pos = startpos + np.cumsum(rng.normal(scale=0.03, size =(N,2)),axis=0)
    time = timestart + pd.to_timedelta(np.cumsum(rng.randint(10,50,size=N)),unit='s')
    posdf = pd.DataFrame(pos,columns=['lat','long'], index=time.round('1s'))
    posdf.index.name='time'
    df_list.append(posdf)
df = pd.concat(df_list,keys=names, names=['Name'])

df=df.reset_index()
df['name_time']=df['Name']+' ' + df['time'].dt.strftime('%d%b-%H:%M:%S')

In [171]:

fig = px.line_mapbox(df, color="Name",hover_data=[df['time']], lat="lat", lon="long",zoom=9, height=600, width=800)
fig.update_traces(line_width=4)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [154]:
df.columns

Index(['Name', 'time', 'lat', 'long', 'name_time'], dtype='object')